In [1]:
import numpy as np
import pandas as pd
import keras.backend as K

Using TensorFlow backend.


In [2]:
img_height = 448
img_width = 448
S = 7
B = 1
C = 2
grid_cell_h = img_height/S
grid_cell_w = img_width/S

In [3]:
a = 0
b = 0
grid_x = []
grid_y = []
for i in range(S):
    grid_x.append([a, a+img_height/S])
    grid_y.append([b, b+img_width/S])
    a += img_height/S
    b += img_width/S

In [17]:
y_pred = np.random.rand(S, S, (5*B+C))
y_true = np.zeros((S, S, 5*B+C))
y_true[6, 0, :] = np.random.rand(7,)
y_true[1, 6, :] = np.random.rand(7,)
y_true[4, 2, :] = np.random.rand(7,)

In [20]:
y_true_ = K.variable(y_true)
y_pred_ = K.variable(y_pred)

In [22]:
def loss():
    pass

In [23]:
def loss_K():
    pass